<a href="https://colab.research.google.com/github/Svarich-11908/machine-learning/blob/main/11_009_Svarich_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ДЗ №1 <a name='dz1'></a>

Набор данных UCI Adult (качать не надо, все есть в репозитории): классификация людей с помощью демографических данных для прогнозирования, зарабатывает ли человек более $ 50 000 в год.

Описание признаков:

Age – возраст, количественный признак
Workclass – тип работодателя, количественный признак
fnlwgt – итоговый вес обьекта, количественный признак
Education – уровень образования, качественный признак
Education_Num – количество лет обучения, количественный признак
Martial_Status – семейное положение, категориальный признак
Occupation – профессия, категориальный признак
Relationship – тип семейных отношений, категориальный признак
Race – раса, категориальный признак
Sex – пол, качественный признак
Capital_Gain – прирост капитала, количественный признак
Capital_Loss – потери капитала, количественный признак
Hours_per_week – количество часов работы в неделю, количественный признак
Country – страна, категориальный признак

Целевая переменная: Target – уровень заработка, категориальный (бинарный) признак

1. Выполнить разведочный анализ данных, заполнить пропуски, преобразовать категориальные данные в числовые:
    - Кодируем категориальные признаки 'Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'. Это можно сделать с помощью метода pandas get_dummies.
    
2. В тестовой выборке не оказалось Голландии. Завести необходимый признак из нулей.

3. Обучить LogisticRegression(). Использовать в качестве baseline.
4. Обучить k-NN, SVC, Naive Bayes, DecisionTree, RandomForest
5. Обучить SVC с такими параметрами {'classifier__gamma':[0.001, 0.01, 0.1, 1, 10, 100], 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]}
6. Обучить DecisionTreeClassifier c параметрами tree_params = {'max_depth': range(2,11)}
7. Обучить RandomForestClassifier c параметрами forest_params = {'max_depth': range(10, 21), 'max_features': range(5, 105, 10)}
8. Использовать кроссвалидацию и GridSearchCV.
9. Сравнить все модели с baseline
10. Вывести confusion matrix

In [3]:

import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, HalvingGridSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ["Age", "Workclass", "fnlwgt", "Education", "Education_Num", "Martial_Status", "Occupation", "Relationship", "Race", "Sex", "Capital_Gain", "Capital_Loss", "Hours_per_week", "Country", "Target"]
data = pd.read_csv(url, names=column_names, sep=',\s*', engine='python')

## Пункт 1. Разведочный анализ <a name='dz1'></a>

In [5]:

categorical_features = ["Workclass", "Education", "Martial_Status", "Occupation", "Relationship", "Race", "Sex", "Country"]
data_encoded = pd.get_dummies(data, columns=categorical_features)
data_encoded

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_?,Workclass_Federal-gov,Workclass_Local-gov,...,Country_Portugal,Country_Puerto-Rico,Country_Scotland,Country_South,Country_Taiwan,Country_Thailand,Country_Trinadad&Tobago,Country_United-States,Country_Vietnam,Country_Yugoslavia
0,39,77516,13,2174,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32557,40,154374,9,0,0,40,>50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32558,58,151910,9,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32559,22,201490,9,0,0,20,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Пункт 2. Голландия <a name='dz1'></a>

In [6]:
data_encoded["From_Holland"] = 0

## Пункт 3. Logistic Regression  <a name='dz1'></a>

In [7]:
X = data_encoded.drop("Target", axis=1)
y = data_encoded["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_accuracy = logistic_model.score(X_test, y_test)
logistic_accuracy2 = logistic_model.score(X_train, y_train)
print("Logistic Regression Accuracy Test:", logistic_accuracy)
print("Logistic Regression Accuracy Train:", logistic_accuracy2)

Logistic Regression Accuracy Test: 0.7996315062183326
Logistic Regression Accuracy Train: 0.7971437346437347


## Пункт 4. Остальные модели  <a name='dz1'></a>

In [8]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_accuracy = knn_model.score(X_test, y_test)
print("k-NN Accuracy:", knn_accuracy)
print('')

svc_model = SVC()
svc_model.fit(X_train, y_train)
svc_accuracy = svc_model.score(X_test, y_test)
print("SVC Accuracy:", svc_accuracy)
print('')


nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_accuracy = nb_model.score(X_test, y_test)
print("Naive Bayes Accuracy:", nb_accuracy)
print('')

dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_accuracy = dt_model.score(X_test, y_test)
print("Decision Tree Accuracy:", dt_accuracy)
print('')

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_accuracy = rf_model.score(X_test, y_test)
print("Random Forest Accuracy:", rf_accuracy)
print('')


k-NN Accuracy: 0.7785966528481498

SVC Accuracy: 0.7956394902502687

Naive Bayes Accuracy: 0.7991708889912482

Decision Tree Accuracy: 0.8185168125287886

Random Forest Accuracy: 0.8590511285122063



## Пункт 5. SVC с параметрами  <a name='dz1'></a>

In [18]:
grid_search_svc = HalvingGridSearchCV(SVC(), param_grid={ 'gamma': [0.001, 0.01, 0.1, 1, 10, 100],'C': [0.001, 0.01, 0.1, 1, 10, 100]}, n_jobs=-1)
grid_search_svc.fit(X_train, y_train)
best_svc_model = grid_search_svc.best_estimator_
best_svc_accuracy = best_svc_model.score(X_test, y_test)
print("Best SVC Accuracy:", best_svc_accuracy)
#Обучение модели занимает много времени

KeyboardInterrupt: ignored

## Пункт 6. DecisionTreeClassifier с параметрами  <a name='dz1'></a>




In [9]:
tree_params = {'max_depth': range(2, 11)}
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5)
grid_search_tree.fit(X_train, y_train)
best_tree_model = grid_search_tree.best_estimator_
best_tree_accuracy = best_tree_model.score(X_test, y_test)
print("Best Decision Tree Accuracy:", best_tree_accuracy)

Best Decision Tree Accuracy: 0.861200675571933


## Пункт 7. RandomForestClassifier с параметрами  <a name='dz1'></a>




In [10]:
forest_params = {'max_depth': range(10, 21), 'max_features': range(5, 105, 10)}
grid_search_forest = GridSearchCV(RandomForestClassifier(), forest_params, cv=5)
grid_search_forest.fit(X_train, y_train)
best_forest_model = grid_search_forest.best_estimator_
best_forest_accuracy = best_forest_model.score(X_test, y_test)
print("Best Random Forest Accuracy:", best_forest_accuracy)
#Обучение модели занимает много времени

KeyboardInterrupt: ignored

## Пункт 8 (уже выполнен) и 9. Сравнение с baseline  <a name='dz1'></a>

In [14]:
print("Baseline Logistic Regression Accuracy:", logistic_accuracy)
print("k-NN Accuracy:", knn_accuracy)
print("SVC Accuracy:", svc_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
#print("Best SVC Accuracy:", best_svc_accuracy)
print("Best Decision Tree Accuracy:", best_tree_accuracy)
#print("Best Random Forest Accuracy:", best_forest_accuracy)

Baseline Logistic Regression Accuracy: 0.7996315062183326
k-NN Accuracy: 0.7785966528481498
SVC Accuracy: 0.7956394902502687
Naive Bayes Accuracy: 0.7991708889912482
Decision Tree Accuracy: 0.8185168125287886
Random Forest Accuracy: 0.8590511285122063
Best Decision Tree Accuracy: 0.861200675571933


## Пункт 10. Вывод Confusion Matrix  <a name='dz1'></a>

In [19]:
#Невозможно без 7-го пункта

### ДЗ №2 <a name='dz1'></a>

## Пункт 1. Деление данных на обучающуюся и тестовую выборку  <a name='dz1'></a>

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

digits = load_digits()

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.15, random_state=42)


## Пункт 2. Создание класификатора и поиск лучших данных <a name='dz1'></a>

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

svm = SVC()


grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_

## Пункт 3. Вывод матрицы несоотвествия  <a name='dz1'></a>

In [24]:
from sklearn.metrics import confusion_matrix

y_pred = best_classifier.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)

print("Матрица несоответствия:")
print(confusion)

Матрица несоответствия:
[[23  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 26  0  0  0  0  0  0  0]
 [ 0  0  0 23  0  1  0  0  0  0]
 [ 0  0  0  0 37  0  0  0  0  0]
 [ 0  0  0  0  0 33  0  0  0  0]
 [ 0  0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0  0 25  0  1]
 [ 0  0  0  0  0  0  0  0 19  0]
 [ 0  0  0  0  0  0  0  1  0 33]]
